In [1]:
import numpy as np
import pandas as pd
import requests
!pip install xlsxwriter
import xlsxwriter
import math


In [2]:
stocks = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AlgoTrading/sp_500_stocks.csv')

In [6]:
#Acquiring API Token from IEX Cloud API token
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [28]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

In [29]:
price = data['latestPrice']
market_cap = data['marketCap']

In [23]:
my_columns = ['Ticker', 'Stock Price', 'Mkt Cap', 'Num Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Mkt Cap,Num Shares to Buy


In [31]:
final_dataframe.append(
    pd.Series(
        [
         symbol,
         price,
         market_cap,
         'N/A'
        ]
    ,index = my_columns,
    ),
    ignore_index=True
)


,Ticker,Stock Price,Mkt Cap,Num Shares to Buy
0,AAPL,124.03,2182423285294,N/A


In [37]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
  api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
  data = requests.get(api_url).json()
  final_dataframe = final_dataframe.append(
      pd.Series([
                 stock,
                 data['latestPrice'],
                 data['marketCap'],
                 'N/A'
      ], index = my_columns),
      True
  )

In [38]:
final_dataframe

,Ticker,Stock Price,Mkt Cap,Num Shares to Buy
0,A,123.66,37291692978,N/A
1,AAL,18.29,10922096765,N/A
2,AAP,161.72,11177526819,N/A
3,AAPL,126.62,2154754047138,N/A
4,ABBV,109.10,193830150526,N/A
...,...,...,...,...
500,YUM,106.60,32727408160,N/A
501,ZBH,148.54,30782920936,N/A
502,ZBRA,378.96,20880383620,N/A
503,ZION,43.48,7090351158,N/A


In [39]:
def chunks(lst, n):
  for i in range(0, len(lst),n):
    yield lst[i:i + n]

In [50]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0,len(symbol_groups)):
  symbol_strings.append(','.join(symbol_groups[i]))
  #symbol_strings = []

final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
  batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
  data = requests.get(batch_api_call_url).json()
  for symbol in symbol_string.split(','):
    final_dataframe = final_dataframe.append(
        pd.Series([
                   symbol,
                   data[symbol]['quote']['latestPrice'],
                   data[symbol]['quote']['marketCap'],
                    'N/A'
        ],index = my_columns),
        ignore_index = True
    )

In [51]:
final_dataframe

,Ticker,Stock Price,Mkt Cap,Num Shares to Buy
0,A,122.29,38382575315,N/A
1,AAL,18.49,11149353826,N/A
2,AAP,163.02,11060247109,N/A
3,AAPL,124.76,2158553122774,N/A
4,ABBV,107.84,194905927031,N/A
...,...,...,...,...
500,YUM,111.37,33628202264,N/A
501,ZBH,153.11,30359308019,N/A
502,ZBRA,376.85,20218149695,N/A
503,ZION,44.10,7172325586,N/A


In [67]:
#Calculating the Number of Shares to Buy
portfolio_size = input('Enter the value of your portfolio ')

try:
  val = float(portfolio_size)
  print(val)
except ValueError:
  print('Please enter an integer')
  portfolio_size = input('Please try again: ')

Enter the value of your portfolio 10000000
10000000.0


In [68]:
position_size = float(val)/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
  final_dataframe.loc[i, 'Num Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
number_of_apple_shares = position_size/500

In [69]:
final_dataframe

,Ticker,Stock Price,Mkt Cap,Num Shares to Buy
0,A,122.29,38382575315,161
1,AAL,18.49,11149353826,1070
2,AAP,163.02,11060247109,121
3,AAPL,124.76,2158553122774,158
4,ABBV,107.84,194905927031,183
...,...,...,...,...
500,YUM,111.37,33628202264,177
501,ZBH,153.11,30359308019,129
502,ZBRA,376.85,20218149695,52
503,ZION,44.10,7172325586,449


In [82]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

In [78]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format' : '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [79]:
writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)

In [83]:
column_formats = {
    'A':['Ticker', string_format],
    'B':['Stock Price', dollar_format],
    'C':['Market Capitalization', dollar_format],
    'D':['Num of Shares', integer_format]
}

In [86]:
for column in column_formats.keys():
  writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
writer.save()

In [87]:
from google.colab import files
files.download('recommended_trades.csv')

FileNotFoundError: ignored